<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Self_Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##MemProtMD Self-Assembly

[MemProtMD](https://doi.org/10.1016/j.str.2015.05.006) membrane protein insertion protocol through self-assembly of a lipid membrane around a protein structure. A database of [PDB](http://rcsb.org) structures can be found [here](http://memprotmd.bioch.ox.ac.uk).

The membrane-spanning regions and protein orientation is predicted using [Memembed](https://doi.org/10.1186/1471-2105-14-276); which may be downloaded from [here](https://github.com/psipred/MemEmbed).

The protein structure is converted to [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) representation using [Martinize](https://github.com/marrink-lab/vermouth-martinize) and run for 60 ns of molecular dynamics simulations using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

The final snapshot of the simulation is converted back to [CHARMM36m](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs) representation using [CG2AT](https://doi.org/10.1021/acs.jctc.1c00295); which can be downloaded from [here](https://github.com/pstansfeld/cg2at).

Inspiration for this Google Colab was taken from the amazing work of the [ColabFold](https://github.com/sokrypton/ColabFold) team.

Change the settings below and click **Runtime → Run all**. 

You will be prompted to either upload a Protein PDB file to simulate. Alternatively, if you select the RCSB you should enter a PDB ID or if the files are from either Swiss-Model or AlphaFold databases enter a UniProtID.

In [ ]:
#@title Initialisation
%%capture
!python3 -m pip install py3dmol
!apt-get update
!apt-get upgrade
!apt-get install gzip

File_Location = "Upload" #@param ["Upload","RCSB PDB","AFDB","Swiss-Model"]

#@markdown ---
#@markdown #### Options for coordinates from databases:
#@markdown #### RCSB:
PDB_ID = "7SSU" #@param {type:"string"}
#@markdown #### AlphaFold or Swiss-Model database:
UniProtID = "P00804" #@param {type:"string"}

In [ ]:
#@title Get PDB coordinate file
import os
import sys

import py3Dmol
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')

if File_Location == "Upload":
  upload = files.upload()
  filename = next(iter(upload))
elif File_Location == "RCSB PDB":
  name = str(PDB_ID.lower())
  os.system('wget http://www.rcsb.org/pdb/files/' + name + '.pdb1.gz')
  os.system('gunzip ' + name + '.pdb1.gz')
  filename = name + '.pdb'
  os.system('egrep -v "MODEL|ENDMDL" '+ name + '.pdb1 > '+ filename)
elif File_Location == "AFDB":
  name = str(UniProtID.upper())
  os.system('wget https://alphafold.ebi.ac.uk/files/AF-' + name + '-F1-model_v2.pdb')
  filename = name + '.pdb'
  os.rename('AF-' + name + '-F1-model_v2.pdb',filename)
elif File_Location == "Swiss-Model":
  name = str(UniProtID.upper())
  os.system('wget https://swissmodel.expasy.org/repository/uniprot/' + name + '.pdb')
  filename = name + '.pdb'

name = os.path.splitext(filename)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + name + '.pdb')
os.chdir(working_dir)
mol1 = open(working_dir + filename, 'r').read()
mview = py3Dmol.view(width=800,height=400) 
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [ ]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/memembed/"):
  !apt-get update -y
  !apt-get install dssp
  !git clone https://github.com/timnugent/memembed
  %cd memembed
  !make
  %cd ../
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import pandas as pd
import shutil

In [ ]:
#@title Energy Minimise & Fix Input Structure
%%capture
if not os.path.exists('temp'):
	os.makedirs('temp')
os.chdir('temp')

os.system('pdb2pqr30 --ff CHARMM  --keep-chain '+ working_dir + filename + ' pqr.pdb')

gromacs.pdb2gmx(f='pqr.pdb',ignh=True,ff='charmm27',water='tip3p',o='conf.pdb')

gromacs.editconf(f='conf.pdb',d=8,c=True,o='conf.pdb')

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')

gromacs.grompp(f='em.mdp',maxwarn=5,o='em',c='conf.pdb')

gromacs.mdrun(deffnm='em',c='clean.pdb')

gromacs.trjconv(f='clean.pdb',o=working_dir + 'fixed-'+filename,s='em.tpr',input=('system'))

os.chdir('..')

shutil.rmtree('temp', ignore_errors=True)

In [ ]:
#@title Predict Membrane Orientation
os.chdir(working_dir)
os.system('/content/memembed/bin/memembed -o memembed.pdb fixed-' + filename) 
mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + '/memembed.pdb', 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [ ]:
#@title Set-up Coarse-Grained Membrane Protein System
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

gromacs.make_ndx(f='memembed.pdb', o='index.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
gromacs.editconf(f='memembed.pdb',o='box.pdb',n='index.ndx',c=True,d=3,input=(0,0),backup=False)

u = MDAnalysis.Universe('box.pdb')
x = round(u.dimensions[0]/10)
y = round(u.dimensions[1]/10)
z = round(u.dimensions[2]/10)
smallbox = [x,y,8]
bigbox = [x,y,z]

gromacs.editconf(f='memembed.pdb',o='centered.pdb',label='A',resnr=1,n='index.ndx',c=True,input=(1,0),backup=False,box=smallbox)

with open('centered.pdb', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('HSE', 'HIS')
filedata = filedata.replace('HSD', 'HIS')
filedata = filedata.replace('MSE', 'MET')
filedata = filedata.replace(' SE ', ' SD ')
with open('centered.pdb', 'w') as file:
  file.write(filedata)

!martinize2 -f centered.pdb  -ff martini3001 -x protein-cg.pdb -o protein-cg.top -elastic -ef 500 -eu 1.0 -el 0.5 -ea 0 -ep 0 -merge A -maxwarn 1000

!sed -e 's/^molecule.*/Protein 1/g' molecule*.itp >  protein-cg.itp

!wget 'http://cgmartini.nl/images/parameters/lipids/PC/POPC/POPC-em.gro'
!wget 'http://cgmartini.nl/images/applications/water/water.gro'

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1')

nlipid = round(x * y * 2.5)

gromacs.editconf(f='protein-cg.pdb', o='protein-box.pdb',label='A',input=(0),backup=False,box=smallbox,noc=True)

gromacs.insert_molecules(f='protein-box.pdb',ci='POPC-em.gro',o='prot+lipid.pdb',nmol=nlipid,radius=0.21,rot='xyz')

v = MDAnalysis.Universe('prot+lipid.pdb')
popc = v.select_atoms('name PO4')
po = popc.atoms.n_atoms

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1\nPOPC ' + str(po))

gromacs.editconf(f='prot+lipid.pdb',o='prot+lipid-box.pdb',c=True,box=bigbox)
gromacs.solvate(cp='prot+lipid-box.pdb',o='prot+lipid+sol.pdb',cs='water.gro',radius=0.21,backup=False,p='topol.top')

v = MDAnalysis.Universe('prot+lipid+sol.pdb')
wat = v.select_atoms('name W')
w = wat.atoms.n_atoms

print(w)

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1\nPOPC ' + str(po) + '\nW ' + str(w))

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 1000\nemstep = 0.001')

gromacs.grompp(f='em.mdp',o='ion.tpr',c='prot+lipid+sol.pdb',maxwarn='-1',backup=False,v=True)
gromacs.genion(s='ion.tpr',o='CG-system.gro',neutral=True,conc=0.15,backup=False,input=str(14),p='topol.top')

v = MDAnalysis.Universe('CG-system.gro')
wat = v.select_atoms('name W')
sod = v.select_atoms('name NA')
chl = v.select_atoms('name CL')
w = wat.atoms.n_atoms
na = sod.atoms.n_atoms
cl = chl.atoms.n_atoms

with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein-cg.itp"\n[ system ]\n[ molecules ]\nProtein 1\nPOPC ' + str(po) + '\nW ' + str(w) + '\nNA ' + str(na) + '\nCL ' + str(cl))

gromacs.grompp(f='em.mdp',o='em.tpr',c='CG-system.gro',maxwarn='-1',backup=False,v=True)

gromacs.mdrun(deffnm='em', c='CG-system.pdb',backup=False)

gromacs.trjconv(f='CG-system.pdb', o='CG-system.pdb', pbc='res', s='em.tpr', conect=True, input='0',backup=False)

%mkdir MD

gromacs.make_ndx(f='CG-system.pdb', o='sys.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),backup=False)

with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 3000000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = c-rescale\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\nlincs_warnangle = 30\n')

gromacs.grompp(f='cgmd.mdp',o='MD/md',c='CG-system.pdb',maxwarn=-1, n='sys.ndx',backup=False)  

mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + 'CG-system.pdb', 'r').read()
mview.addModel(mol1,'gro')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'resn':'POPC'},{'sphere':{}})
mview.setStyle({'atom':'BB'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

for file in glob.glob(r'#*'):
    os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [ ]:
#@title Equilibrate Membrane Protein System
os.chdir(working_dir + 'MD')
gromacs.mdrun(deffnm='md',backup=False,v=True,resethway=True,nstlist=100)
mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + 'MD/md.gro', 'r').read()
mview.addModel(mol1,'gro')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'atom':'PO4'},{'sphere':{}})
mview.setStyle({'atom':'BB'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

for file in glob.glob(r'#*'):
    os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [ ]:
#@title Convert back to Atomistic
os.chdir(working_dir)
shutil.rmtree('Atomistic', ignore_errors=True)
os.chdir(working_dir + 'MD/')
shutil.rmtree('CG2AT', ignore_errors=True)
os.system('/content/cg2at/cg2at -group all -o align -w tip3p -c md.gro -a ../fixed-'+name+'.pdb -loc CG2AT -ff charmm36-jul2020-updated -fg martini_3-0_charmm36')

os.chdir(working_dir + 'MD/CG2AT/FINAL/')
gromacs.grompp(f='../../../em.mdp', o='final.tpr', c='final_cg2at_aligned.pdb', p='topol_final.top',maxwarn=-1)

shutil.copytree(working_dir + 'MD/CG2AT/FINAL/', working_dir + 'Atomistic/')
os.chdir(working_dir + 'Atomistic/')
gromacs.editconf(f='final.tpr',o=name + '-System.pdb')

os.rename(working_dir + 'MD/md.gro', working_dir + name + '-cgmd.gro')
os.rename(working_dir + 'MD/md.tpr', working_dir + name + '-cgmd.tpr')
os.rename(working_dir + 'memembed.pdb', working_dir + name + '-memembed.pdb')

p1 = MDAnalysis.Universe('../fixed-'+ name +'.pdb')
p1residues = p1.select_atoms('name CA').resids
start = p1residues[0]

gromacs.editconf(f=name +'-System.pdb',o=name +'-System.pdb',resnr=int(start))
p2 = MDAnalysis.Universe(name +'-System.pdb')
p2residues = p2.select_atoms('name CA').resids

All = p2.atoms

df = pd.DataFrame({'original':p2residues.tolist(),'new':p1residues.tolist()})

df = df.set_index('original')

for i in All:
        if i.residue.resid in df.index:
                i.residue.resid = df.loc[i.residue.resid,'new']

All.write(name +"-System.pdb")

os.rename(working_dir + 'Atomistic/topol_final.top', working_dir + 'Atomistic/topol.top')
gromacs.make_ndx(f=working_dir + 'Atomistic/' + name + '-System.pdb', o='index.ndx', input=('del 2-40', 'rSOL|rNA*|rCL*','1|2','0&!3','del 3','name 2 water_and_ions','name 3 Lipid','q'),backup=False)
with open(working_dir + 'Atomistic/topol.top', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('../FINAL/', '')
with open(working_dir + 'Atomistic/topol.top', 'w') as file:
  file.write(filedata)
!wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/500ns.mdp
!wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/main/mdp_files/10ns-pr.mdp
mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + 'Atomistic/' + name + '-System.pdb', 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'atom':'P'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [ ]:
#@title Download Zip
os.chdir(working_dir)
shutil.rmtree('MD', ignore_errors=True)
try:
    os.remove('em*.trr')
except OSError:
    pass
try:
    os.remove('em*.tpr')
except OSError:
    pass
try:
    os.remove('em*.log')
except OSError:
    pass
try:
    os.remove('protein.pdb')
except OSError:
    pass
try:
    os.remove('molecule_0.itp')
except OSError:
    pass
try:
    os.remove('mdout.mdp')
except OSError:
    pass
try:
    os.remove('aligned.gro')
except OSError:
    pass
try:
    os.remove('protein-cg.top')
except OSError:
    pass
os.chdir('/content/')
os.system('zip -r ' + name + '.zip ' + name )
files.download(name + '.zip')